In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random # parser function
import glob

In [2]:
tf.reset_default_graph()

In [3]:
BATCH_SIZE = 50
NUM_EPOCHS = 100
LOG_DIR = '.graph'

In [4]:
def parser(filepath):
    image_string = tf.read_file(filepath)
    
    # Don't use tf.image.decode_image, or the output shape will be undefined
    image = tf.image.decode_jpeg(image_string, channels=3)

    # This will convert to float values in [0, 1]
    image = tf.image.convert_image_dtype(image, tf.float32)

    image = tf.image.resize_images(image, [120, 120])

    return image

In [5]:
bamp = 0.4
cmin = 0.1
cmax = 0.5
hamp = 0.1 # 0-0.5
smin = 0.1
smax = 0.3
transformations = [\
                   lambda x, y: (tf.image.random_flip_up_down(x), y),\
                   lambda x, y: (tf.image.random_flip_left_right(x), y),\
                   lambda x, y: (tf.image.transpose_image(x), y),\
                   lambda x, y: (tf.image.random_brightness(x, bamp), y),\
                   lambda x, y: (tf.image.random_contrast(x, cmin, cmax), y),\
                   lambda x, y: (tf.image.random_hue(x, hamp), y),\
                   lambda x, y: (tf.image.random_saturation(x, smin, smax), y)]


In [6]:
def extract_label(filepath):
    label = filepath.split('/')[1]
    return 1 if label=='cat' else 0

In [7]:
# change how to split path for label if path is changed
path = 'train_small/*/*.jpg'
filepath = glob.glob(path)

images = list(map(lambda x: parser(x), filepath))
label  = list(map(lambda x: extract_label(x), filepath))

# dataset = tf.data.Dataset.list_files(path, shuffle=True)
# # dataset = tf.data.TFRecordDataset(files)
# dataset = dataset.map(parser)

dataset = tf.data.Dataset.from_tensor_slices((images, label))
dataset = dataset.shuffle(2000)
for i in range(len(transformations)):
    augment = dataset.map(transformations[i])
    dataset.concatenate(augment)
    print("Finished Transformation " + str(i))
dataset = dataset.map(lambda x, y: (tf.clip_by_value(x, 0.0, 1.0), y))
dataset = dataset.repeat(NUM_EPOCHS)
print("Cloned for number of epochs")
dataset = dataset.batch(BATCH_SIZE)
print("batching finished")
dataset = dataset.prefetch(1)

iterator = dataset.make_one_shot_iterator()
features, labels = iterator.get_next()

Finished Transformation 0
Finished Transformation 1
Finished Transformation 2
Finished Transformation 3
Finished Transformation 4
Finished Transformation 5
Finished Transformation 6
Cloned for number of epochs
batching finished


In [8]:
# with tf.Session() as sess:
#     img = sess.run(features)
#     label = sess.run(labels)
#     batchlen = len(label)
#     ncols = 5
#     nrows = int(np.ceil(batchlen/ncols))
#     fig, ax = plt.subplots(nrows=nrows, ncols=ncols)
#     fig.set_figwidth(16)
#     fig.set_figheight(nrows*3)
#     for i in range(batchlen):
#         a, b = i//ncols, i%ncols
#         ax[a, b].imshow(img[i])
#         ax[a, b].axis('off')
#         ax[a, b].set_title(str(label[i]))

In [9]:
is_training = True
vgg = tf.layers.conv2d(inputs=features,filters=16, kernel_size=[3,3], padding='SAME', activation=tf.nn.relu)
vgg = tf.layers.conv2d(inputs=vgg,filters=16, kernel_size=[3,3], padding='SAME', activation=tf.nn.relu)
vgg = tf.layers.max_pooling2d(inputs=vgg, pool_size=[2,2], strides=2)

vgg = tf.layers.flatten(vgg)

vgg = tf.layers.dense(vgg, units=512, activation=tf.nn.relu)
vgg = tf.layers.dropout(vgg, rate=0.5, training=is_training)

# vgg = tf.layers.dense(vgg, units=100, activation=tf.nn.relu)
# vgg = tf.layers.dropout(vgg, rate=0.5, training=is_training)

logits = tf.layers.dense(vgg, units=1, activation=tf.nn.relu)

predictions = {
  "classes": tf.argmax(input=logits, axis=1),
  "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
}

# Calculate Loss
loss = tf.losses.softmax_cross_entropy(tf.reshape(labels, [-1,1]), logits)
xent = tf.reduce_mean(loss)

# Configure the Training 
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
train_step = optimizer.minimize(
    loss=xent,
    global_step=tf.train.get_global_step())

# Add evaluation metrics
eval_metric = {
    "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}

tf.summary.scalar('cross_entropy', xent, )
tf.summary.scalar('accuracy', eval_metric["accuracy"][0])
summ = tf.summary.merge_all()

In [10]:
counter = 0
with tf.Session() as sess:
    init_g = tf.global_variables_initializer()
    init_l = tf.local_variables_initializer()
    writer = tf.summary.FileWriter(LOG_DIR, sess.graph)
    saver  = tf.train.Saver()
    sess.run(init_g)
    sess.run(init_l)
    for epoch in range(NUM_EPOCHS):
            [train_accuracy, s] = sess.run([eval_metric["accuracy"], summ])
            writer.add_summary(s, counter)
            print("Epoch: %2d, training accuracy : %6.5f"%(epoch, train_accuracy[0]))
            sess.run(train_step)
            counter = counter+1

Epoch:  0, training accuracy : 0.00000
Epoch:  1, training accuracy : 0.48000
Epoch:  2, training accuracy : 0.46000
Epoch:  3, training accuracy : 0.52000
Epoch:  4, training accuracy : 0.51500
Epoch:  5, training accuracy : 0.52800
Epoch:  6, training accuracy : 0.51667
Epoch:  7, training accuracy : 0.50286
Epoch:  8, training accuracy : 0.49250
Epoch:  9, training accuracy : 0.48444
Epoch: 10, training accuracy : 0.47200
Epoch: 11, training accuracy : 0.47818
Epoch: 12, training accuracy : 0.48500
Epoch: 13, training accuracy : 0.49231
Epoch: 14, training accuracy : 0.49571
Epoch: 15, training accuracy : 0.49067
Epoch: 16, training accuracy : 0.49125
Epoch: 17, training accuracy : 0.48824
Epoch: 18, training accuracy : 0.48333
Epoch: 19, training accuracy : 0.48316
Epoch: 20, training accuracy : 0.48100
Epoch: 21, training accuracy : 0.48000
Epoch: 22, training accuracy : 0.47818
Epoch: 23, training accuracy : 0.48000
Epoch: 24, training accuracy : 0.48250
Epoch: 25, training accur